In [4]:
import json
from openai import OpenAI
import os

# Set up the OpenAI API client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
DEPLOYMENT_NAME = 'gpt-4o'

# Example function hard coded to return the same weather


# Function to square a number
def get_square_number(number: int) -> int:
    """Retruns square of a number"""
    return number**2

def run_conversation():
    messages = [
        { 
            "role": "system", 
            "content": """
                You are a helpful assistant.
                You have access to a function that can square a number.
            """
        }
    ]
    while True:
        user_input = input("User: ")
        if user_input.lower() in ['exit', 'quit', 'stop']:
            break
        messages.append({"role": "user", "content": user_input})
        tools = [
            {
                "type": "function",
                "function": {
                    "name": "get_square_number",
                    "description": """
                        Returns square of a given number. 
                    """,
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "number": {
                                "type": "integer",
                                "description": "The input number that need to be squared",
                            },
                        },
                        "required": ["number"],
                    },
                },
            }
        ]
        response = client.chat.completions.create(
            model=DEPLOYMENT_NAME,
            messages=messages,
            tools=tools,
            tool_choice="auto",
            temperature=0,
        )
        response_message = response.choices[0].message
        tool_calls = response_message.tool_calls
        if tool_calls:
            messages.append(response_message)
            for tool_call in tool_calls:
                messages.append(
                    {
                        "tool_call_id": tool_call.id,
                        "role": "tool",
                        "name": str(tool_call.function.name),
                        "content": str(get_square_number(**json.loads(tool_call.function.arguments))),
                    }
                )
            second_response = client.chat.completions.create(
                model=DEPLOYMENT_NAME,
                messages=messages,
                temperature=0,
            )
            print("Assistant:", second_response.choices[0].message['content'])
        else:
            print("Assistant:", response_message['content'])

run_conversation()


ChatCompletion(id='chatcmpl-Ajd8eMNzWlIRufT7f4gGvejyBxhsq', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The square of 834 is 695,556.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1735438572, model='gpt-4o-2024-08-06', object='chat.completion', service_tier=None, system_fingerprint='fp_d28bcae782', usage=CompletionUsage(completion_tokens=12, prompt_tokens=66, total_tokens=78, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))